<a href="https://colab.research.google.com/github/lame/CarPriceTracker/blob/master/PorscheScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import datetime
import pickle
import re
import dateutil
import time

In [0]:
base_url = 'https://bringatrailer.com'
auction_results_url = 'auctions/results'

# TODO: You may want to use a user-agent that is not `python-requests-` to not
# trigger anti-bot measures
request = requests.get('/'.join([base_url, auction_results_url])).text
soup = bs(request)

In [0]:
for auction_result in soup.find('div', id='initial-results'):
  a = auction_result.find('a')
  title = a.text
  link = a['href']
  print(title, link)

In [0]:
no_reserve_re = re.compile('[Nn]o [Rr]eserve')
year_re = re.compile('[\d]{4}')
make_model_trim_re = re.compile('\d{4}.*')
tmu_re = re.compile('TMU')
lot_re = re.compile('Lot')
mileage_re = re.compile('[\d]*k?\s[Mm]iles')
location_re = re.compile('Location:')

def re_search(regex, text):
  search = regex.search(text)
  return search.group(0) if search is not None else None

def get_year(text):
  return re_search(year_re, text)

def get_make_model_trim(text):
  result = re_search(make_model_trim_re, text)
  return result.split(' ')[1:] if result else None

def is_no_reserve(text):
  return bool(re_search(no_reserve_re, text))

def to_date(date_str):
  date_str = re.match('[\w\s,]+', date_str)[0]
  return dateutil.parser.parse(date_str)

def to_int(number_str):
  number_str = re.match('[\d,$]+', number_str)[0]
  number_str = number_str.replace('$', '').replace(',', '')
  return int(number_str)

def tmu(text):
  return bool(re_search(tmu_re, text))

def mileage(text):
  result = re_search(mileage_re, text)
  return to_int(search.group(0).split(' ')[0].replace('k', '000')) if result else None

def location(tag):
  postal_code = country = None
  _, city, state, *args = listing_essentials.find(text=location_re).replace(',', '').replace(':', '').split()
  if args[0].isdigit():
    postal_code = to_int(args[0])
    country = 'USA'
  else:
    counrty = args[0]
  return {
      'city': city, 
      'state': state, 
      'country': country, 
      'postal_code': postal_code,
  }

def lot_num(tag):
  return to_int(listing_essentials.find(text=lot_re).split('#')[-1])

In [0]:
listings = []
auction_items = soup.find('div', id='initial-results').find_all('div', class_='auctions-item-extended')

for item in auction_items:
  auction_item = {}
  
  title = item.find('span').text.strip() # Title
  status = item.find('div', class_='auctions-item-status').text

  auction_item['link'] = item.find('a')['href'] # Link
  auction_item['title'] = title
  auction_item['no_reserve'] = is_no_reserve(title)
  auction_item['model_year'] = get_year(title)
  auction_item['make_model_trim'] = get_make_model_trim(title)
  auction_item['auction_date'] = to_date(status.split()[-1])
  auction_item['price'] = to_int(status.split()[2])
  auction_item['sold'] = status.split()[0].lower() == 'sold'
  listings.append(auction_item)

In [0]:
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)
pp.pprint(listings[:4])

[   {   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
        'link': 'https://bringatrailer.com/listing/2000-bmw-m-roadster-58/',
        'make_model_trim': ['BMW', 'M', 'Roadster'],
        'model_year': '2000',
        'no_reserve': True,
        'price': 16250,
        'sold': True,
        'title': 'No Reserve: 2000 BMW M Roadster'},
    {   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
        'link': 'https://bringatrailer.com/listing/1991-jaguar-xjs-23/',
        'make_model_trim': ['Jaguar', 'XJS', 'Convertible'],
        'model_year': '1991',
        'no_reserve': False,
        'price': 16500,
        'sold': True,
        'title': '19k-Mile 1991 Jaguar XJS Convertible'},
    {   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
        'link': 'https://bringatrailer.com/listing/1999-porsche-911-carrera-119/',
        'make_model_trim': [   'Porsche',
                               '911',
                               'Carrera',
                            

In [0]:
len(items)

32

In [0]:
for listing in listings:
  listing_request = requests.get(listing['link']).text
  listing_soup = bs(listing_request)

  listing_essentials = listing_soup.find('div', class_='listing-essentials')
  listing['lot_number'] = lot_num(listing_essentials)
  listing['tmu'] = tmu(listing_essentials.text)
  listing['mileage'] = mileage(listing_essentials.text)
  listing['location'] = location(listing_essentials)

  pp.pprint(listing)
  time.sleep(1)

{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2000-bmw-m-roadster-58/',
    'location': {   'city': 'Beaverton',
                    'country': 'USA',
                    'postal_code': 97007,
                    'state': 'Oregon'},
    'lot_number': 30983,
    'make_model_trim': ['BMW', 'M', 'Roadster'],
    'mileage': 50000,
    'model_year': '2000',
    'no_reserve': True,
    'price': 16250,
    'sold': True,
    'title': 'No Reserve: 2000 BMW M Roadster',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1991-jaguar-xjs-23/',
    'location': {   'city': 'Raleigh',
                    'country': None,
                    'postal_code': None,
                    'state': 'North'},
    'lot_number': 30980,
    'make_model_trim': ['Jaguar', 'XJS', 'Convertible'],
    'mileage': 50000,
    'model_year': '1991',
    'no_reserve': False,
    'price': 16500,
  

In [0]:
listings = {   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2000-bmw-m-roadster-58/',
    'location': {   'city': 'Beaverton',
                    'country': 'USA',
                    'postal_code': 97007,
                    'state': 'Oregon'},
    'lot_number': 30983,
    'make_model_trim': ['BMW', 'M', 'Roadster'],
    'mileage': 50000,
    'model_year': '2000',
    'no_reserve': True,
    'price': 16250,
    'sold': True,
    'title': 'No Reserve: 2000 BMW M Roadster',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1991-jaguar-xjs-23/',
    'location': {   'city': 'Raleigh',
                    'country': None,
                    'postal_code': None,
                    'state': 'North'},
    'lot_number': 30980,
    'make_model_trim': ['Jaguar', 'XJS', 'Convertible'],
    'mileage': 50000,
    'model_year': '1991',
    'no_reserve': False,
    'price': 16500,
    'sold': True,
    'title': '19k-Mile 1991 Jaguar XJS Convertible',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1999-porsche-911-carrera-119/',
    'location': {   'city': 'Denver',
                    'country': 'USA',
                    'postal_code': 80216,
                    'state': 'Colorado'},
    'lot_number': 30979,
    'make_model_trim': ['Porsche', '911', 'Carrera', 'Cabriolet', '6-Speed'],
    'mileage': 50000,
    'model_year': '1999',
    'no_reserve': False,
    'price': 16100,
    'sold': True,
    'title': '1999 Porsche 911 Carrera Cabriolet 6-Speed',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1970-honda-trail-90-2/',
    'location': {   'city': 'Portland',
                    'country': 'USA',
                    'postal_code': 97218,
                    'state': 'Oregon'},
    'lot_number': 30985,
    'make_model_trim': ['Honda', 'Trail', '90'],
    'mileage': 50000,
    'model_year': '1970',
    'no_reserve': True,
    'price': 2800,
    'sold': True,
    'title': 'No Reserve: 1970 Honda Trail 90',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1969-chevrolet-corvette-56/',
    'location': {   'city': 'Monterey',
                    'country': 'USA',
                    'postal_code': 93940,
                    'state': 'California'},
    'lot_number': 30986,
    'make_model_trim': [   'Chevrolet',
                           'Corvette',
                           'Convertible',
                           '350',
                           '5-Speed'],
    'mileage': 50000,
    'model_year': '1969',
    'no_reserve': False,
    'price': 22701,
    'sold': True,
    'title': '1969 Chevrolet Corvette Convertible 350 5-Speed',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2006-mercedes-benz-sl600-6/',
    'location': {   'city': 'Naples',
                    'country': 'USA',
                    'postal_code': 34119,
                    'state': 'Florida'},
    'lot_number': 30976,
    'make_model_trim': ['Mercedes-Benz', 'SL600'],
    'mileage': 50000,
    'model_year': '2006',
    'no_reserve': False,
    'price': 23750,
    'sold': True,
    'title': '39k-Mile 2006 Mercedes-Benz SL600',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2006-porsche-911-carrera-4-6/',
    'location': {   'city': 'Miami',
                    'country': 'USA',
                    'postal_code': 33186,
                    'state': 'Florida'},
    'lot_number': 30984,
    'make_model_trim': ['Porsche', '911', 'Carrera', '4'],
    'mileage': 50000,
    'model_year': '2006',
    'no_reserve': True,
    'price': 26750,
    'sold': True,
    'title': 'No Reserve: 2006 Porsche 911 Carrera 4',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2000-jaguar-xkr-11/',
    'location': {   'city': 'Ball',
                    'country': None,
                    'postal_code': None,
                    'state': 'Ground'},
    'lot_number': 30967,
    'make_model_trim': ['Jaguar', 'XKR', 'Convertible'],
    'mileage': 50000,
    'model_year': '2000',
    'no_reserve': False,
    'price': 31750,
    'sold': True,
    'title': 'Modified 2000 Jaguar XKR Convertible',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1969-honda-mini-trail-50-8/',
    'location': {   'city': 'Allendale',
                    'country': 'USA',
                    'postal_code': 49401,
                    'state': 'Michigan'},
    'lot_number': 30982,
    'make_model_trim': ['Honda', 'Z50A', 'Monkey'],
    'mileage': None,
    'model_year': '1969',
    'no_reserve': True,
    'price': 4800,
    'sold': True,
    'title': 'No Reserve: 1969 Honda Z50A Monkey',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2004-porsche-911-turbo-cabriolet-41/',
    'location': {   'city': 'Upland',
                    'country': 'USA',
                    'postal_code': 91784,
                    'state': 'California'},
    'lot_number': 30981,
    'make_model_trim': ['Porsche', '911', 'Turbo', 'Cabriolet'],
    'mileage': None,
    'model_year': '2004',
    'no_reserve': False,
    'price': 47250,
    'sold': True,
    'title': '29k-Mile 2004 Porsche 911 Turbo Cabriolet',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1972-ford-f-100-11/',
    'location': {   'city': 'Los',
                    'country': None,
                    'postal_code': None,
                    'state': 'Angeles'},
    'lot_number': 30978,
    'make_model_trim': ['Ford', 'F-100'],
    'mileage': 50000,
    'model_year': '1972',
    'no_reserve': False,
    'price': 9999,
    'sold': False,
    'title': '1972 Ford F-100',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1985-audi-ur-quattro-12/',
    'location': {   'city': 'Mount',
                    'country': None,
                    'postal_code': None,
                    'state': 'Kisco'},
    'lot_number': 30977,
    'make_model_trim': ['Audi', 'Ur-Quattro'],
    'mileage': 50000,
    'model_year': '1985',
    'no_reserve': False,
    'price': 49100,
    'sold': True,
    'title': '1985 Audi Ur-Quattro',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1961-austin-healey-bug-eye-sprite-14/',
    'location': {   'city': 'Johns',
                    'country': None,
                    'postal_code': None,
                    'state': 'Island'},
    'lot_number': 30973,
    'make_model_trim': ['Austin-Healey', 'Bugeye', 'Sprite'],
    'mileage': 50000,
    'model_year': '1961',
    'no_reserve': True,
    'price': 18750,
    'sold': True,
    'title': 'No Reserve: 1961 Austin-Healey Bugeye Sprite',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2014-audi-rs-5-13/',
    'location': {   'city': 'Sanford',
                    'country': 'USA',
                    'postal_code': 32771,
                    'state': 'Florida'},
    'lot_number': 30972,
    'make_model_trim': ['Audi', 'RS5'],
    'mileage': 50000,
    'model_year': '2014',
    'no_reserve': False,
    'price': 40500,
    'sold': True,
    'title': '11k-Mile 2014 Audi RS5',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2002-honda-s2000-63/',
    'location': {   'city': 'Tallahassee',
                    'country': 'USA',
                    'postal_code': 32317,
                    'state': 'Florida'},
    'lot_number': 30974,
    'make_model_trim': ['Honda', 'S2000'],
    'mileage': 50000,
    'model_year': '2002',
    'no_reserve': False,
    'price': 19000,
    'sold': True,
    'title': '2002 Honda S2000',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1980-puch-maxi-s-2-speed/',
    'location': {   'city': 'Pleasanton',
                    'country': 'USA',
                    'postal_code': 94566,
                    'state': 'California'},
    'lot_number': 30971,
    'make_model_trim': ['Puch', 'Maxi', 'Sport', 'Mk', 'II'],
    'mileage': 50000,
    'model_year': '1980',
    'no_reserve': True,
    'price': 5800,
    'sold': True,
    'title': 'No Reserve: 1980 Puch Maxi Sport Mk II',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1923-ford-t-bucket-11-2/',
    'location': {   'city': 'Erie',
                    'country': 'USA',
                    'postal_code': 16509,
                    'state': 'Pennsylvania'},
    'lot_number': 30970,
    'make_model_trim': ['Ford', 'T-Bucket', 'Hot', 'Rod'],
    'mileage': 50000,
    'model_year': '1923',
    'no_reserve': False,
    'price': 22500,
    'sold': True,
    'title': '1923 Ford T-Bucket Hot Rod',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2010-porsche-911-gt3-rs-8/',
    'location': {   'city': 'Mill',
                    'country': None,
                    'postal_code': None,
                    'state': 'Valley'},
    'lot_number': 30962,
    'make_model_trim': ['Porsche', '911', 'GT3', 'RS'],
    'mileage': 50000,
    'model_year': '2010',
    'no_reserve': False,
    'price': 126500,
    'sold': True,
    'title': '2010 Porsche 911 GT3 RS',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/scuderia-ferrari-annual-yearbook/',
    'location': {   'city': 'San',
                    'country': None,
                    'postal_code': None,
                    'state': 'Rafael'},
    'lot_number': 30961,
    'make_model_trim': ['Scuderia', 'Ferrari', 'Yearbook'],
    'mileage': None,
    'model_year': '1932',
    'no_reserve': False,
    'price': 5200,
    'sold': True,
    'title': '1932 Scuderia Ferrari Yearbook',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1977-land-rover-88-series-iii/',
    'location': {   'city': 'Boston',
                    'country': 'USA',
                    'postal_code': 2116,
                    'state': 'Massachusetts'},
    'lot_number': 30964,
    'make_model_trim': ['Land', 'Rover', '88', 'Series', 'III'],
    'mileage': 50000,
    'model_year': '1977',
    'no_reserve': True,
    'price': 12000,
    'sold': True,
    'title': 'No Reserve: 1977 Land Rover 88 Series III',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2009-porsche-cayenne-turbo-s-11/',
    'location': {   'city': 'Fort',
                    'country': None,
                    'postal_code': None,
                    'state': 'Lauderdale'},
    'lot_number': 30956,
    'make_model_trim': ['Porsche', 'Cayenne', 'Turbo', 'S'],
    'mileage': 50000,
    'model_year': '2009',
    'no_reserve': False,
    'price': 29000,
    'sold': True,
    'title': '43k-Mile 2009 Porsche Cayenne Turbo S',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2007-bentley-continental-gt-6/',
    'location': {   'city': 'Akron',
                    'country': 'USA',
                    'postal_code': 44313,
                    'state': 'Ohio'},
    'lot_number': 30965,
    'make_model_trim': ['Bentley', 'Continental', 'GT'],
    'mileage': 50000,
    'model_year': '2007',
    'no_reserve': False,
    'price': 41200,
    'sold': False,
    'title': '2007 Bentley Continental GT',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1995-mitsubishi-pajero-mini/',
    'location': {   'city': 'Cookeville',
                    'country': 'USA',
                    'postal_code': 38501,
                    'state': 'Tennessee'},
    'lot_number': 30953,
    'make_model_trim': ['Mitsubishi', 'Pajero', 'Mini', 'XR-II'],
    'mileage': 50000,
    'model_year': '1995',
    'no_reserve': True,
    'price': 14250,
    'sold': True,
    'title': 'No Reserve: 11k-Mile 1995 Mitsubishi Pajero Mini XR-II',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2008-bmw-535-40/',
    'location': {   'city': 'Milton',
                    'country': 'USA',
                    'postal_code': 2186,
                    'state': 'Massachusetts'},
    'lot_number': 30959,
    'make_model_trim': ['BMW', '535xi', 'Sports', 'Wagon'],
    'mileage': 50000,
    'model_year': '2008',
    'no_reserve': True,
    'price': 12850,
    'sold': True,
    'title': 'No Reserve: 2008 BMW 535xi Sports Wagon',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1966-bug-rear-engine-sprint/',
    'location': {   'city': 'Vancouver',
                    'country': 'USA',
                    'postal_code': 98685,
                    'state': 'Washington'},
    'lot_number': 30955,
    'make_model_trim': ['Bug', 'Sprint', 'Kart'],
    'mileage': None,
    'model_year': '1966',
    'no_reserve': True,
    'price': 7000,
    'sold': True,
    'title': 'No Reserve: 1966 Bug Sprint Kart',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1998-land-rover-range-rover-2/',
    'location': {   'city': 'Northbrook',
                    'country': 'USA',
                    'postal_code': 60062,
                    'state': 'Illinois'},
    'lot_number': 30960,
    'make_model_trim': ['Land', 'Rover', 'Range', 'Rover', '4.6', 'HSE'],
    'mileage': 50000,
    'model_year': '1998',
    'no_reserve': False,
    'price': 12000,
    'sold': True,
    'title': '1998 Land Rover Range Rover 4.6 HSE',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2020-chevy-corvette/',
    'location': {   'city': 'Delavan',
                    'country': 'USA',
                    'postal_code': 53115,
                    'state': 'Wisconsin'},
    'lot_number': 30954,
    'make_model_trim': ['Chevrolet', 'Corvette', 'Stingray', 'Coupe'],
    'mileage': 50000,
    'model_year': '2020',
    'no_reserve': False,
    'price': 91500,
    'sold': True,
    'title': '2020 Chevrolet Corvette Stingray Coupe',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1948-lincoln-continental-coupe-2/',
    'location': {   'city': 'Kingwood',
                    'country': 'USA',
                    'postal_code': 77345,
                    'state': 'Texas'},
    'lot_number': 30947,
    'make_model_trim': ['Lincoln', 'Continental', 'Coupe'],
    'mileage': 50000,
    'model_year': '1948',
    'no_reserve': False,
    'price': 32000,
    'sold': True,
    'title': '1948 Lincoln Continental Coupe',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/2012-porsche-cayman-r-16/',
    'location': {   'city': 'Campbell',
                    'country': 'USA',
                    'postal_code': 95008,
                    'state': 'California'},
    'lot_number': 30950,
    'make_model_trim': ['Porsche', 'Cayman', 'R', '6-Speed'],
    'mileage': 50000,
    'model_year': '2012',
    'no_reserve': False,
    'price': 46300,
    'sold': True,
    'title': '2012 Porsche Cayman R 6-Speed',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1983-ford-mustang-gt-5-0-5/',
    'location': {   'city': 'Topton',
                    'country': 'USA',
                    'postal_code': 19562,
                    'state': 'Pennsylvania'},
    'lot_number': 30948,
    'make_model_trim': ['Ford', 'Mustang', 'GT', '5.0', '4-Speed'],
    'mileage': 50000,
    'model_year': '1983',
    'no_reserve': False,
    'price': 16000,
    'sold': True,
    'title': '36-Years-Owned 1983 Ford Mustang GT 5.0 4-Speed',
    'tmu': False}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1960-mercedes-benz-190sl-18/',
    'location': {   'city': 'Ball',
                    'country': None,
                    'postal_code': None,
                    'state': 'Ground'},
    'lot_number': 30949,
    'make_model_trim': ['Mercedes-Benz', '190SL'],
    'mileage': 50000,
    'model_year': '1960',
    'no_reserve': False,
    'price': 69100,
    'sold': True,
    'title': '1960 Mercedes-Benz 190SL',
    'tmu': True}
{   'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
    'link': 'https://bringatrailer.com/listing/1972-honda-z600-9/',
    'location': {   'city': 'Ventura',
                    'country': 'USA',
                    'postal_code': 93001,
                    'state': 'California'},
    'lot_number': 30951,
    'make_model_trim': ['Honda', 'Z600'],
    'mileage': 50000,
    'model_year': '1972',
    'no_reserve': False,
    'price': 11250,
    'sold': True,
    'title': '1972 Honda Z600',
    'tmu': True}

    

{'auction_date': datetime.datetime(2020, 5, 5, 0, 0),
 'link': 'https://bringatrailer.com/listing/1972-honda-z600-9/',
 'location': {'city': 'Ventura',
  'country': 'USA',
  'postal_code': 93001,
  'state': 'California'},
 'lot_number': 30951,
 'make_model_trim': ['Honda', 'Z600'],
 'mileage': 50000,
 'model_year': '1972',
 'no_reserve': False,
 'price': 11250,
 'sold': True,
 'title': '1972 Honda Z600',
 'tmu': True}

In [0]:
import pickle
with open('listings.pickle', 'wb') as file:
  pickle.dump(listings, file)

In [0]:
# Previously Listed

listing_html = requests.get('https://bringatrailer.com/listing/2006-bmw-m3-coupe-41/').text
soup = bs(listing_html)

In [0]:
soup.find('div', class_='post-excerpt').find_all('a', rel='noreferrer')

[<a href="https://bringatrailer.com/listing/2006-bmw-m3-coupe-20/" rel="noopener noreferrer" target="_blank">previously listed on BaT in 2018</a>]